In [44]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap, jitter


In [43]:
api = wandb.Api()
runs = api.runs("hmr24/FundusDRGrading-src_fundusClassif_scripts")

important_keys = ['lr', 'ema', 'swa', 'mixup', 'preprocessing', 'as_regression']
list_configs = []
for i, r in enumerate(runs):
    kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
    if kappa is None:
        continue
    config = {k: v for k, v in r.config.items() if k in important_keys}
    config['name'] = r.name
    config['kappa'] = kappa
    list_configs.append(config)
    
    if i >= 50:
        break
    

In [47]:
df = pd.DataFrame(list_configs)
datasource = ColumnDataSource(df)
output_notebook()

xrange = ['Without EMA', 'With EMA']
p = figure(height=500, title="Kappa values")

p.scatter(jitter('ema', 0.05), y='kappa', source=datasource, size=10, color="blue")
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1
show(p)

Loading BokehJS ...

{'lr': 3.407674268842685e-05,
 'ema': 0,
 'swa': 1,
 'mixup': 1,
 'define': [512, 512],
 'optimizer': 'AdamW',
 'mixup_alpha': 0.2750413850688639,
 'training/lr': 3.407674268842685e-05,
 'cutmix_alpha': 0.4213267053431159,
 'datasets/ddr': '/home/florian/Documents/donnees_medicales/DDR-dataset/DR_grading/',
 'as_regression': 0,
 'data/img_size': [512, 512],
 'preprocessing': 'autobalance',
 'data/drop_last': True,
 'data/use_cache': True,
 'datasets/aptos': '/home/florian/Documents/donnees_medicales/aptos/',
 'datasets/idrid': '/home/florian/Documents/donnees_medicales/IDRID/B. Disease Grading/',
 'logger/project': 'Grading-DiabeticRetinopathy-Comparisons-V3',
 'model/img_size': [512, 512],
 'data/batch_size': 48,
 'data/valid_size': 0.2,
 'trainer/devices': [0],
 'data/num_workers': 8,
 'datasets/eyepacs': '/home/florian/Documents/donnees_medicales/eyepacs/',
 'model/pretrained': True,
 'data/cache_option': 'disk',
 'model/num_classes': 5,
 'trainer/precision': '16-mixed',
 'model/arc